In [ ]:
%%capture
!pip install adapters accelerate bitsandbytes datasets torch

# import required

In [ ]:
import torch
import math
from huggingface_hub import login
from datasets import load_dataset
from transformers import set_seed
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from adapters import init, AdapterConfig
from adapters import AdapterTrainer
from adapters.composition import Stack
from transformers import DataCollatorForLanguageModeling

# Training domain adapter with quantized Llama 3.2 - 1B model

Adapterhub notebook on finetuning Llama QLoRA used as reference:

https://github.com/adapter-hub/adapters/blob/main/notebooks/QLoRA_Llama_Finetuning.ipynb

In [ ]:
SEED = 42
HF_KEY = 'hf_key'
datasetpath = 'hf_dataset_path'
modelpath = 'base_model_path'
adapter_type = 'pfeiffer'
r_factor = 16
output_dir = 'output_dir'
adapter_upload_path = 'adapter_upload_path'
adapter_save_path = 'adapter_save_path'
domain_adapter_path = 'domain_adapter_path'

lr = 0.0005
num_epochs = 1

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
login(HF_KEY) # login to save checkpoints/models/adapters to HF hub
set_seed(42)
torch.manual_seed(42)
qa_dataset = load_dataset(datasetpath, split="train") # load dataset from HF hub
qa_dataset.shuffle(42)
qa_dataset = qa_dataset.train_test_split(test_size=0.15) # split dataset into train and test

In [ ]:
# Load 4-bit quantized model
model = AutoModelForCausalLM.from_pretrained(
    modelpath,
    device_map="auto",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
    ),
    torch_dtype=torch.bfloat16,
)
model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(modelpath)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
init(model) # when not using AdapterModel directly
model.load_adapter(domain_adapter_path, load_as='domain', with_head=False) # load trained domain adapter either from local path or HF hub

config = AdapterConfig.load(adapter_type, reduction_factor=r_factor)
model.add_adapter("task", config=config)
model.train_adapter("task")
model.active_adapters = Stack('domain', 'task')

model.adapter_to(device)

print(model.adapter_summary())

In [ ]:
# casting parameters for quantized setting
for param in model.parameters():
    if param.ndim == 1:
        # cast the small parameters (e.g. layernorm) to fp32 for stability
        param.data = param.data.to(torch.float32)

# if needed, to enable gradient checkpointing to reduce required memory
# model.gradient_checkpointing_enable()
# model.enable_input_require_grads()

class CastOutputToFloat(torch.nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

In [ ]:
qa_dataset = qa_dataset.map(lambda x: {"text": f"Answer the following question based on your biomedical knowledge: {x['input']} \nAnswer: {x['output']}"})

def tokenize(element):
    return tokenizer(element["text"],
                     truncation=True,
                     max_length=256,
                     add_special_tokens=False,)

column_names = qa_dataset["train"].column_names
qa_data = qa_dataset.map(tokenize, batched=True, remove_columns=column_names)

In [ ]:
args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    num_train_epochs=num_epochs,
    logging_steps=1000,
    #save_steps=500,
    #eval_steps=187,
    #save_total_limit=3,
    #gradient_accumulation_steps=16,
    #max_steps=1875,
    lr_scheduler_type="constant",
    optim="paged_adamw_32bit",
    learning_rate=lr,
    #group_by_length=True,
    bf16=True,
    report_to="none",
    #warmup_ratio=0.03,
    #max_grad_norm=0.3,
)

trainer = AdapterTrainer(
    model=model,
    tokenizer=tokenizer,
    data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False),
    train_dataset = qa_data["train"],
    eval_dataset =  qa_data["test"],
    args=args,
)

In [ ]:
trainer.train()

In [ ]:
eval_res = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_res['eval_loss']):.2f}")

In [ ]:
model.push_adapter_to_hub(adapter_upload_path, "task") # to upload adapter modules
# model.save_adapter(adapter_save_path, 'task') # or saving locally